# SAR 
* v2 data - 모델링 실패 ㅠㅠ 
* 한 명의 유저가 여러개의 평점을 매겨야 하는데, 14만 유저가 평점 하나씩만 매겼기 때문에 .....
* user-item transpose는 말이 안 됨! - 한 개의 item에 해당하는 top-k user 추천? no no no

## 1. Load Data

In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', 50)

In [2]:
import json
from pandas.io.json import json_normalize

In [ ]:
import zipfile

def unzip(source_file, dest_path):
    with zipfile.ZipFile(source_file, 'r') as zf:
        zf.extractall(path=dest_path)
        zf.close()

In [ ]:
unzip(os.path.join("..","..","data","TA_User_Review_Seoul_all_v2.json.zip"), os.getcwd())

In [3]:
with open("TA_User_Review_Seoul_all_v2.json", 'r') as f:
    df_json = json.load(f)

In [4]:
df_json[0]

{'__typename': 'Review',
 'createdDate': '2020-03-06',
 'helpfulVotes': 0,
 'id': 749418557,
 'language': 'ja',
 'location.__typename': 'LocationInformation',
 'location.additionalNames.long': 'Hotel Gracery Seoul, South Korea',
 'location.additionalNames.longOnlyParent': 'South Korea',
 'location.locationId': 14159911,
 'location.name': 'Hotel Gracery Seoul',
 'location.parent.additionalNames.long': 'Seoul, South Korea',
 'location.parent.locationId': 294197,
 'location.parentGeoId': 294197,
 'location.placeType': 'ACCOMMODATION',
 'photoIds': [],
 'photos': [],
 'rating': 4,
 'socialStatistics.followCount': 0,
 'socialStatistics.isFollowing': False,
 'socialStatistics.isLiked': False,
 'socialStatistics.isReposted': False,
 'socialStatistics.isSaved': False,
 'socialStatistics.likeCount': 0,
 'socialStatistics.repostCount': 0,
 'socialStatistics.tripCount': 0,
 'text': '日系のホテルなので日本語OK\n南大門も明洞も歩いて行けます\nトリプルルーム\u3000お部屋は狭めで3人分の荷物を広げるのは少し窮屈でしたが\nお風呂が日本式の独立したお風呂なのてゆっくりできました😊\n\n',
 'titl

In [5]:
df = json_normalize(df_json)

In [6]:
df.head()

,__typename,createdDate,helpfulVotes,id,language,location.__typename,location.additionalNames.long,location.additionalNames.longOnlyParent,location.locationId,location.name,location.parent.additionalNames.long,location.parent.locationId,location.parentGeoId,location.placeType,photoIds,photos,rating,socialStatistics.followCount,socialStatistics.isFollowing,socialStatistics.isLiked,socialStatistics.isReposted,socialStatistics.isSaved,socialStatistics.likeCount,socialStatistics.repostCount,socialStatistics.tripCount,text,title,userId,userProfile.contributionCounts.sumAllUgc,userProfile.isMe,userProfile.isVerified,userProfile.userId
0,Review,2020-03-06,0,749418557,ja,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,"Seoul, South Korea",294197,294197,ACCOMMODATION,[],[],4,0,False,False,False,False,0,0,0,日系のホテルなので日本語OK\n南大門も明洞も歩いて行けます\nトリプルルーム お部屋は狭め...,グレイスリーホテルソウル,E44FFBC7D1E0A97FECCCF7F8EB052E0C,8.0,0.0,0.0,E44FFBC7D1E0A97FECCCF7F8EB052E0C
1,Review,2020-03-01,0,748454819,en,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,"Seoul, South Korea",294197,294197,ACCOMMODATION,[453807126],"[{'id': 453807126, 'statuses': ['PENDING'], 'p...",5,0,False,False,False,False,0,0,0,I really enjoyed our stay here in Gracery Hote...,Great Hotel,F9AD0C1F6409A1574AD0AFC237D2A4CE,7.0,0.0,0.0,F9AD0C1F6409A1574AD0AFC237D2A4CE
2,Review,2020-02-16,0,745371386,ja,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,"Seoul, South Korea",294197,294197,ACCOMMODATION,[],[],5,0,False,False,False,False,0,0,0,今回はagoda経由で予約しました。予約後に日系のホテルと知った次第です。\nチェックイン後...,安心出来ます,770775D59B126E71D38D56FAB9B95F93,57.0,0.0,0.0,770775D59B126E71D38D56FAB9B95F93
3,Review,2020-02-11,0,744426202,ja,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,"Seoul, South Korea",294197,294197,ACCOMMODATION,[],[],5,0,False,False,False,False,0,0,0,こちらのホテルは夫婦で何度か利用させていただいてます。立地もいいですが、お風呂とトイレが別な...,やはり落ち着く,4F1BF9403CA4EB27756E0C824A088553,43.0,0.0,0.0,4F1BF9403CA4EB27756E0C824A088553
4,Review,2020-02-05,0,743141310,en,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,"Seoul, South Korea",294197,294197,ACCOMMODATION,[],[],5,0,False,False,False,False,0,0,0,Its about 7 minutes walking distance to City H...,"Ask for higher floor, it has better city view",23EF5413495FF530151DB7E5C0FEF38D,225.0,0.0,0.0,23EF5413495FF530151DB7E5C0FEF38D


In [7]:
COLUMNS = ['id', 'location.locationId', 'location.name', 'rating', 'createdDate']
df = df.loc[:,COLUMNS]
df.head()

,id,location.locationId,location.name,rating,createdDate
0,749418557,14159911,Hotel Gracery Seoul,4,2020-03-06
1,748454819,14159911,Hotel Gracery Seoul,5,2020-03-01
2,745371386,14159911,Hotel Gracery Seoul,5,2020-02-16
3,744426202,14159911,Hotel Gracery Seoul,5,2020-02-11
4,743141310,14159911,Hotel Gracery Seoul,5,2020-02-05


## 2. git
* https://github.com/microsoft/recommenders/tree/master/reco_utils/recommender/sar

In [7]:
pip install git+https://github.com/microsoft/recommenders 

  Cloning https://github.com/microsoft/recommenders to /tmp/pip-req-build-274lohze
  Running command git clone -q https://github.com/microsoft/recommenders /tmp/pip-req-build-274lohze
  Created wheel for reco-utils: filename=reco_utils-2019.9-cp36-none-any.whl size=164194 sha256=484bb1fe7bd422d8ce383bdf476ab3128647cbbad3f0e8cb856726388390c7aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-bc_u9k95/wheels/b4/1c/70/c3f6005781b2b5a3370ec55bdd75c10ed1b07456bdfb1a0524
Successfully built reco-utils


In [8]:
import sys
sys.path.append("../../")

import itertools
import logging
import os

import papermill as pm

from reco_utils.dataset.python_splitters import python_stratified_split, python_random_split, python_chrono_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

In [9]:
import datetime
import time 

In [10]:
df.groupby("id").agg({"location.locationId": "count"}).describe() # 다 이게 1이라서 제대로 train/test 안나뉘어지는듯 ... 

,location.locationId
count,144434.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [11]:
df.groupby('location.locationId').agg({'id': "count"}).describe() # 우리 데이터에는 이런식으로 들어가야 함 ㅠㅠ 

,id
count,6712.000000
mean,21.518772
std,124.781167
min,1.000000
25%,2.000000
50%,4.000000
75%,11.000000
max,3986.000000


## 3. Data Preprocessing / 4. Modeling

In [12]:
# top k items to recommend
TOP_K = 10
DATA_SIZE = df.shape[0]

In [13]:
# timestamp 형식으로 바꿔주기 
df["createdDate"] = df.apply(lambda x : time.mktime(datetime.datetime.strptime(x["createdDate"], "%Y-%m-%d").timetuple()), axis=1)

In [14]:
df.head()

,id,location.locationId,location.name,rating,createdDate
0,749418557,14159911,Hotel Gracery Seoul,4,1.583420e+09
1,748454819,14159911,Hotel Gracery Seoul,5,1.582988e+09
2,745371386,14159911,Hotel Gracery Seoul,5,1.581779e+09
3,744426202,14159911,Hotel Gracery Seoul,5,1.581347e+09
4,743141310,14159911,Hotel Gracery Seoul,5,1.580828e+09


In [15]:
df[["id"]].nunique()

id    144434
dtype: int64

In [16]:
df[["location.locationId"]].nunique()

location.locationId    6712
dtype: int64

In [ ]:
# train / test split code 
def _do_stratification(
    data,
    ratio=0.75,
    min_rating=1,
    filter_by="user",
    is_random=True,
    seed=42,
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    col_timestamp=DEFAULT_TIMESTAMP_COL,
):
    # A few preliminary checks.
    if not (filter_by == "user" or filter_by == "item"):
        raise ValueError("filter_by should be either 'user' or 'item'.")

    if min_rating < 1:
        raise ValueError("min_rating should be integer and larger than or equal to 1.")

    if col_user not in data.columns:
        raise ValueError("Schema of data not valid. Missing User Col")

    if col_item not in data.columns:
        raise ValueError("Schema of data not valid. Missing Item Col")

    if not is_random:
        if col_timestamp not in data.columns:
            raise ValueError("Schema of data not valid. Missing Timestamp Col")

    multi_split, ratio = process_split_ratio(ratio)

    split_by_column = col_user if filter_by == "user" else col_item

    ratio = ratio if multi_split else [ratio, 1 - ratio]

    if min_rating > 1:
        data = min_rating_filter_pandas(
            data,
            min_rating=min_rating,
            filter_by=filter_by,
            col_user=col_user,
            col_item=col_item,
        )

    # Split by each group and aggregate splits together.
    splits = []

    # If it is for chronological splitting, the split will be performed in a random way.
    ### 여기서 그룹으로 묶을 때 !!! 
    ### data.sort_values(timestamp).groupby(userId)로 묶고 시작하게 됨 
    df_grouped = (
        data.sort_values(col_timestamp).groupby(split_by_column) 
        if is_random is False
        else data.groupby(split_by_column)
    )

    for name, group in df_grouped:
        group_splits = split_pandas_data_with_ratios(
            df_grouped.get_group(name), ratio, shuffle=is_random, seed=seed
        )

        # Concatenate the list of split dataframes.
        concat_group_splits = pd.concat(group_splits)

        splits.append(concat_group_splits)

    # Concatenate splits for all the groups together.
    splits_all = pd.concat(splits)

    # Take split by split_index
    splits_list = [
        splits_all[splits_all["split_index"] == x].drop("split_index", axis=1)
        for x in range(len(ratio))
    ]

    return splits_list

In [ ]:
def stratified_split(
    data,
    ratio=0.75,
    min_rating=1,
    filter_by="user",
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    seed=42,
):
    """Pandas stratified splitter.
    
    For each user / item, the split function takes proportions of ratings which is
    specified by the split ratio(s). The split is stratified.

    Args:
        data (pd.DataFrame): Pandas DataFrame to be split.
        ratio (float or list): Ratio for splitting data. If it is a single float number
            it splits data into two halves and the ratio argument indicates the ratio of
            training data set; if it is a list of float numbers, the splitter splits
            data into several portions corresponding to the split ratios. If a list is
            provided and the ratios are not summed to 1, they will be normalized.
        seed (int): Seed.
        min_rating (int): minimum number of ratings for user or item.
        filter_by (str): either "user" or "item", depending on which of the two is to
            filter with min_rating.
        col_user (str): column name of user IDs.
        col_item (str): column name of item IDs.

    Returns:
        list: Splits of the input data as pd.DataFrame.
    """
    return _do_stratification(
        data,
        ratio=ratio,
        min_rating=min_rating,
        filter_by=filter_by,
        col_user=col_user,
        col_item=col_item,
        is_random=True,
        seed=seed,
    )

### 1) A matrix: (user, item) - stratified_split

In [17]:
# 모델을 살펴보니 transpose 안 해주는게 맞는 것 같음  
header = {
    "col_user": "id",
    "col_item": "location.locationId",
    "col_rating": "rating",
    "col_timestamp": "createdDate",
}

In [19]:
train, test = python_stratified_split(df, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed = 42 ) 

In [20]:
print(train.shape, test.shape) # user=id, item=locationId 넣으면 groupby 묶는 것 때문에 train test = (14만, ), (0, ) 으로 나뉘게 됨 

(144434, 5) (0, 5)


### 2) A matrix : (user, item) - random split

In [21]:
header = {
    "col_user": "id",
    "col_item": "location.locationId",
    "col_rating": "rating",
    "col_timestamp": "createdDate",
}

In [23]:
train, test = python_random_split(df, ratio=0.75, seed = 42)

In [24]:
print(train.shape, test.shape) 

(108325, 5) (36109, 5)


In [25]:
model = SARSingleNode(
    similarity_type="cooccurrence", # jaccard, lift, cooccurrence 
    time_decay_coefficient=100, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

In [26]:
model.fit(train)

In [27]:
top_k = model.recommend_k_items(test, remove_seen=True) # ^^... 

ValueError: SAR cannot score users that are not in the training set

### 3) A matrix : (item, user) 

In [28]:
header = {
    "col_user": "location.locationId",
    "col_item": "id",
    "col_rating": "rating",
    "col_timestamp": "createdDate",
}

In [29]:
train, test = python_stratified_split(df, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed = 42 ) 

In [30]:
print(train.shape, test.shape) # user=locationId, item=userId 넣고 train test split 한 것 (matrix transpose)

(108958, 5) (35476, 5)


#### (1) cooccurrence

In [31]:
model = SARSingleNode(
    similarity_type="cooccurrence", # jaccard, lift, cooccurrence 
    time_decay_coefficient=100, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

In [32]:
model.fit(train)

In [33]:
top_k = model.recommend_k_items(test, remove_seen=True)

In [34]:
display(top_k.head(10))

,location.locationId,id,prediction
0,299152,735790106,0.0
1,299152,730572264,0.0
2,299152,430028964,0.0
3,299152,660273151,0.0
4,299152,725896100,0.0
5,299152,163731518,0.0
6,299152,395270536,0.0
7,299152,485512510,0.0
8,299152,147941546,0.0
9,299152,753581121,0.0


In [35]:
top_k.loc[top_k["prediction"]!=0] ## ㅋㅋmuzi...? 

,location.locationId,id,prediction


In [37]:
# all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(col_user='location.locationId', 
              col_item='id', 
              col_rating='rating', 
              col_prediction='prediction', 
              relevancy_method='top_k', 
              k=TOP_K)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

In [38]:
print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')

Model:
Top K:		 10
MAP:		 0.000000
NDCG:		 0.000000
Precision@K:	 0.000000
Recall@K:	 0.000000


#### (2) jaccard / lift

In [41]:
model = SARSingleNode(
    similarity_type="lift", # jaccard, lift, cooccurrence 
    time_decay_coefficient=100, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

In [42]:
model.fit(train) # MemoryError 

MemoryError: Unable to allocate 88.5 GiB for an array with shape (108958, 108958) and data type float64

In [0]:
top_k = model.recommend_k_items(test, remove_seen=True)

In [0]:
display(top_k.head(20))

In [0]:
# all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(col_user='location.locationId', 
              col_item='id', 
              col_rating='rating', 
              col_prediction='prediction', 
              relevancy_method='top_k', 
              k=TOP_K)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

In [0]:
print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')